In [1]:
import pandas as pd
import numpy as np
print(f"numpy                {np.__version__:<20}")
print(f"pandas               {pd.__version__:<20}")

import pytest

np.random.seed(42)

numpy                1.19.2              
pandas               1.1.3               


CRRA utility: 

- Suppose you are offered a coin flip for \$2, how much would you pay?
- Probably no more than \$1
- Probably a bit less than \$1, reflecting a risk premium. You demand a higher return for taking the risk.
- Suppose you are offered a retirement annuity of \$10,000 per year, how much would you pay? Probably about the NPV using some discount rate
- Suppose you are offered a variable annuity that draws the payment from some distribution. How much would you pay? Probably a little less: The NPV using a slightly higher discount rate reflecting a risk premium.
- A related question is how much of your net worth you would invest in a positive EV bet. If your preference is to always invest the same share of your net worth, you exhibit constant relative risk aversion, or CRRA. It just means you don't become more or less risk averse as you become more wealthy. The risk premium you demand for a given bet that is a given fraction of your wealth always remains the same.
- If you tell me you the amount of the risk premium you demand, and that you always invest the same fraction of your net worth, I can tell you that your utility function looks like the below and also what your $\gamma$ risk aversion parameter is:

1) $u(c) = \frac{c^{1-\gamma} -1}{1-\gamma}$

- Using this formula, I can derive your risk premium or equivalently the amount you would pay for the variable annuity. If your utility is given by the formula above, I can calculate the equivalent constant payment stream as the inverse function

2) $c = {(1 + u(c) (1 - \gamma))}^\frac{1}{1 -\gamma}$

To value a variable income stream, I convert it to utility using formula 1 and my risk aversion parameter, then convert the utility to the equivalent constant stream of cash using formula 2). Then I can discount that stream at the risk free rate and that tells me what the variable risk stream is worth.

https://en.wikipedia.org/wiki/Isoelastic_utility

In [2]:
# so here's how I think about this utility function. 
# utility is a slightly fraught topic.
# in school I was taught that you shouldn't think about cardinal utility, only ordinal utility. 
# i.e. people can rank preferences but you can't sum utilities, especially across people.
# but if an individual can rank their preferences using indifference curves, there is always 
# some cardinal utility function that will give the same result. you can just count how many indifference curves
# separate any alternatives from a base case, i.e. integrate across the isocurves.
# here are some reasons you might not want to think that way
# base case is arbitrary
# you can't sum utilities, i.e. complementarity comes into play
# the form of the utility function is controversial, once you choose a utility function, you've largely determined
# the nature of the economic 'game' and proper policy. (As a for-instance, if you believe all people should count equally
# in the social welfare function and individuals have decreasing marginal utility of wealth or income, you will tend
# to think equality is a social good. This offends some people. It's not enough to say more equality is better, but 
# there are tradeoffs with efficiency, liberty, allowing a free people a full measure of self-realization. Economic
# 'neutrality' means saying you're entitled to believe social welfare is fully subjective and you shouldn't worry
# about equality and it's a legitimate subjective choice to ignore some people's welfare entirely).
# 
# you might think this utility function is unrealistic and complicated and abstract. and that's OK. just plug in 
# your own version. at the end of the day you have to optimize something. to me the CRRA utility is the absolute
# rock bottom simplest thing you can maximize, and it will take into acount risk tolerance in a reasonable way.

# my gut is that a human being's relative risk tolerance decreases as they grow more wealthy, as implied by Maslow's 
# hierarchy of needs. It starts high. if your wealth and income and status falls below some minimum temporarily,
# it is very unpleasant and you can have permanent social, health repercussions. increased stress, lose social status, 
# you are at risk of homelessness, loss of shelter, health care, food. And even if you have more than you need
# each consecutve doubling probably yields some more or less constant positive desirability, even for the richest
# mogul or king, suggesting log utility. If you're Elon Musk and you have $100b, and you bet according to Kelly,
# you maximize your growth rate and you have a 50% chance of at some point losing 50% of your wealth, 1% chance of
# of losing 99% of your wealth, which would leave you with $1b and still in the very long run almost certainly better
# off.

# the bottom line is, you have to maximize something, and this is the simplest thing I can think of that makes sense.
# be literally dead. 


In [3]:
# monte carlo like this is generally a pretty bad way to model financial time series
# not all the things that can happen did happen
# what did happen didn't happen according to its long-term distribution (sampling error). 
# you could estimate the long term distribution and sample from it.  i.e. generate random log normal data per the correlation matrix. but there is no generally accepted form of the distribution. log normal only a poor approximation. 
# the distribution changes, the real economy and financial structure are totally different from 90 years ago with gold standard, manufacturing/agrarian, less globally integrated supply chains and financial system, different taxes, financial system, politics and wars
# financial time series are serially correlated. if you have a crash in October, due annual averaging, 
# it will impact the current and the following year. financial time series discretize decisions that 
# sometimes play out over multiple time periods, sometimes occur multiple times in a single time period. 
# sometimes you have data anomalies that reverse the following period.


In [4]:
# turtles all the way down

# maybe growth will continue, we'll harness clean energy, eventually build a Dyson sphere
# maybe we'll hit limits to growth and blow up
# maybe we'll just continue to have punctuated equilibria and evolution

# you want to do something that never failed in history, and also has acceptable failure under monte carlo. they will always be that monte carlo trial that has 5 1929s in a row and it's not surivivable.

# be conservative because things are worse, than expected, even taking into account murphy's law.


In [5]:
# compute CE cash flow with a mortality curve

# compute % of original cohort that dies each year
# mortality starts at 1% in year one and goes up linearly by 1% each year
mortality = np.linspace(0.00, 1.0, 101)
survivorship = np.cumprod(1-mortality[:31])
deathrate = -np.diff(survivorship)
# top up last 1 to get to 
deathrate[-1] += 1- np.sum(deathrate)
print ("sum", np.sum(deathrate))
deathrate



sum 1.0


array([0.01      , 0.0198    , 0.029106  , 0.03764376, 0.04517251,
       0.05149666, 0.05647467, 0.06002451, 0.06212537, 0.06281565,
       0.06218749, 0.0603784 , 0.05756074, 0.05392999, 0.04969263,
       0.04505465, 0.04021128, 0.03533862, 0.03058754, 0.0260799 ,
       0.02190712, 0.01813075, 0.0147848 , 0.01187927, 0.00940442,
       0.00733545, 0.00563701, 0.00426742, 0.00318228, 0.0077911 ])

In [6]:
# random cashflows
cashflows = np.random.uniform(1, 100, 30)
cashflows

array([38.07947177, 95.12071633, 73.46740024, 60.26718994, 16.4458454 ,
       16.44345751,  6.7502776 , 86.75143843, 60.51038616, 71.0991852 ,
        3.03786494, 97.02107536, 83.41182144, 22.02157196, 19.00067175,
       19.15704648, 31.11998205, 52.95088673, 43.76255685, 29.83168488,
       61.57343658, 14.8098922 , 29.9223202 , 37.26982249, 46.15092844,
       78.73242018, 20.76770443, 51.9092094 , 59.64904232,  5.59859086])

In [7]:
# utility, for example log utility (gamma = 1)
u = np.log(cashflows)
u

array([3.63967534, 4.55514678, 4.29684177, 4.09878784, 2.80007289,
       2.79992768, 1.90958363, 4.463047  , 4.10281502, 4.26407588,
       1.11115494, 4.57492823, 4.42379004, 3.09202252, 2.94447433,
       2.95267061, 3.43785012, 3.96936482, 3.77877859, 3.39557108,
       4.12023055, 2.69529535, 3.3986047 , 3.61818395, 3.83191708,
       4.36605502, 3.03339911, 3.94949622, 4.08847809, 1.72251493])

In [8]:
indices = np.indices(u.shape)[0]+1
indices

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [9]:
# cumulative mean utility
mean_u = np.cumsum(u)/indices
mean_u

array([3.63967534, 4.09741106, 4.16388797, 4.14761293, 3.87810492,
       3.69840872, 3.44286228, 3.57038537, 3.62954422, 3.69299738,
       3.45828443, 3.55133808, 3.61844977, 3.58084783, 3.53842293,
       3.50181341, 3.49805086, 3.52423497, 3.537632  , 3.53052896,
       3.55860998, 3.51936841, 3.51411781, 3.5184539 , 3.53099243,
       3.56311022, 3.54349129, 3.55799147, 3.57628411, 3.5144918 ])

In [10]:
# convert utility back to cash flow and multiply by number of years for cumulative ce cash flow
ce = np.exp(mean_u) * indices
ce

array([  38.07947177,  120.3685446 ,  192.96334598,  253.13103899,
        241.66267216,  242.2979546 ,  218.93446207,  284.24226157,
        339.26068512,  401.65056775,  349.38683112,  418.31918156,
        484.63650235,  502.65555566,  516.18907746,  530.80893156,
        561.86645922,  610.70054169,  653.32256476,  682.84044909,
        737.40142762,  742.78814031,  772.48456618,  809.57363536,
        853.94625157,  917.09105093,  933.86151596,  982.59396472,
       1036.47409588, 1007.96546332])

In [11]:
# experienced cumulative CE cash flow 
# multiply each year's % who died in that year by cumulative CE cash flow up to that year
# and then sum for average experienced total cash flow
madj_ce = np.sum(ce * deathrate)
madj_ce

# (there's a (weak) argument for using the average, 
# i.e. adjusting to equal weight the guy who died in year 1 with the guy who died in year 30)


436.58452788516126

In [12]:
# that is the CE value of 1 cohort's experienced outcome
# if I have a strategy and I want the CE value of the whole history of all cohorts
# take the CE value experience of each cohort
# model the value of the whole history as if you were drawing the experience of one cohort at random 
# so take the CE value of all the CE values (i.e. discount the risk not only within each cohort but over all cohorts)
# and that's the value of your comprehensive retirement strategy, adjusted for risk

In [13]:
def crra_ce(cashflows, gamma):
    """takes a numpy array, returns total CRRA certainty-equivalent cash flow"""
    # calibration factor to make the total CE cash flow for Bengen rule == 0.1, to reduce numerical problems
    # for gamma > 1, there is an upper bound to utility 1/(gamma-1)
    # when utility approaches this limit, small improvements in cash flow don't numerically change utility
    # when you multiply cash flow by a factor, CE cash flow is multiplied by same factor, optimality is invariant
    # but we convert units back before returning so units in/out are invariant
    calibration_factor = len(cashflows) * 4 * 10
    cashflows = cashflows/calibration_factor
    
    # for retirement study assume no negative cashflows
    if np.any(np.where(cashflows < 0, 1, 0)):
        return 0.0
    elif gamma >= 1.0 and 0 in cashflows:
        return 0.0
    elif gamma == 1.0:
        # general formula for CRRA utility undefined for gamma = 1 but limit as gamma->1 = log
        u = np.mean(np.log(cashflows))
        ce = np.exp(u)
    elif gamma == 2.0:  # simple optimization
        u2 = np.mean(1 - 1.0 / cashflows)
        ce = 1.0 / (1.0 - u2)
    elif gamma > 4.0:
        # force computations as longdouble for more precision, but always return np.float
        gamma = np.longdouble(gamma)
        cashflows = cashflows.astype(np.longdouble)
        gamma_m1 = gamma - 1.0
        gamma_m1_inverse = 1.0 / gamma_m1
        u = np.mean(gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1))
        ce = 1.0 / (1.0 - gamma_m1 * u) ** gamma_m1_inverse
        ce = np.float(ce)
    elif gamma > 1.0:
        gamma_m1 = gamma - 1.0
        gamma_m1_inverse = 1.0 / gamma_m1
        u = np.mean(gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1))
        ce = 1.0 / (1.0 - gamma_m1 * u) ** gamma_m1_inverse
    else:  # general formula
        g_1m = 1 - gamma
        u = np.mean((cashflows ** g_1m - 1.0) / g_1m)
        ce = (g_1m * u + 1.0) ** (1.0 / g_1m)

    return ce * len(cashflows) * calibration_factor

In [14]:
def crra_ce_deathrate(cashflows, gamma, deathrate):
    """ce cash flow with a mortality curve
    cash flows = real cash flows in each year of cohort
    gamma = risk aversion
    death rate = % of cohort that died in each year of cohort
    
    compute utility of each cash flow under gamma
    compute cumulative mean of utilities up to each year
    convert utilities back to CE cash flows
    each member of cohort that died in a given year experienced CE cash flow * years alive
    """
    # for retirement study assume no negative cashflows
    if np.any(np.where(cashflows < 0, 1, 0)):
        return 0.0
    else:
        calibration_factor = len(cashflows) * 4 * 10
        cashflows = cashflows/calibration_factor
        
        # 1..lastyear
        indices = np.indices(cashflows.shape)[0]+1
        
        if gamma == 1.0:
            # utility
            u = np.log(cashflows)
            # cumulative mean utility
            u_mean = np.cumsum(u)/indices
            # cumulative mean ce cash flows
            ce = np.exp(u_mean) * indices
        elif gamma == 2.0:  # simple optimization
            u2 = 1 - 1.0 / cashflows
            u2_mean = np.cumsum(u2)/indices
            ce = 1.0 / (1.0 - u2_mean) * indices
        elif gamma > 4.0:
            # force computations as longdouble for more precision, but always return np.float
            gamma = np.longdouble(gamma)
            cashflows = cashflows.astype(np.longdouble)
            gamma_m1 = gamma - 1.0
            gamma_m1_inverse = 1.0 / gamma_m1
            u = gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1)
            u_mean = np.cumsum(u)/indices
            ce = 1.0 / (1.0 - gamma_m1 * u_mean) ** gamma_m1_inverse
            ce = (ce * indices).astype(float)
        elif gamma > 1.0:
            gamma_m1 = gamma - 1.0
            gamma_m1_inverse = 1.0 / gamma_m1
            u = gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1)
            u_mean = np.cumsum(u)/indices
            ce = 1.0 / (1.0 - gamma_m1 * u_mean) ** gamma_m1_inverse
            ce = ce * indices
        else:  # general formula
            gamma_1m = 1 - gamma
            u = (cashflows ** gamma_1m - 1.0) / gamma_1m
            u_mean = np.cumsum(u)/indices
            ce = (gamma_1m * u_mean + 1.0) ** (1.0 / gamma_1m)
            ce = ce * indices
        # mortality_adjusted ce cash flows
        madj_ce = np.sum(ce * deathrate)
        return madj_ce * calibration_factor


In [15]:
# test 0 and -1
testseries = np.random.uniform(1, 100, 100)
testseries[50] = 0.0
assert crra_ce(testseries, 1) == 0, "bad value"
testseries[50] = -1.0
assert crra_ce(testseries, 1) == 0, "bad value"


In [17]:
# a constant series should always be sum of cash flows for any gamma
testseries = np.ones(100)
for gamma in [0, 0.5, 1, 2, 4, 8, 16]:
    print(gamma, crra_ce(testseries, gamma))
    assert crra_ce(testseries, gamma) == pytest.approx(np.sum(testseries), 0.0000001)

0 99.99999999985576
0.5 99.9999999999992
1 99.99999999999983
2 100.0
4 100.00000000000004
8 100.0
16 100.0


In [18]:
testseries = np.random.uniform(1, 100, 100)

def general_ce(cashflows, gamma):
    cashflows = np.longdouble(cashflows)
    if gamma == 1:
        u = np.mean(np.log(cashflows))
        ce = np.exp(u)
    else:
        u = np.mean((cashflows ** (1-gamma) -1) / (1-gamma))
        ce = (1 + u * (1-gamma)) ** (1/(1-gamma))
    ce = np.float(ce)
    return ce * len(cashflows)

# can't go to 16 without some numerical problems
for gamma in [0, 0.5, 1, 2, 4, 8, 15]:
    print(gamma, crra_ce(testseries, gamma), general_ce(testseries, gamma))
    assert crra_ce(testseries, gamma) == pytest.approx(general_ce(testseries, gamma), 0.000001)


0 4913.441232636373 4913.441232636358
0.5 4393.680124057666 4393.68012405766
1 3734.955584397617 3734.9555843976204
2 2056.766184643597 2056.7661846435963
4 575.7431234707183 575.743123470718
8 282.21136564731336 282.21136564731336
15 208.07151484074058 208.07151484074055


In [19]:
# if everyone dies in last year, mortality-adjusted CE = unadjusted CE
deathrate = np.zeros(100)
deathrate[99] = 1.0

# can't go too high without some numerical problems
for gamma in [0, 0.5, 1, 2, 4, 8, 16]:
    print(gamma, crra_ce(testseries, gamma), crra_ce_deathrate(testseries, gamma, deathrate))
    assert crra_ce(testseries, gamma) == pytest.approx(general_ce(testseries, gamma), 0.000001)


0 4913.441232636373 4913.441232636195
0.5 4393.680124057666 4393.680124057666
1 3734.955584397617 3734.955584397623
2 2056.766184643597 2056.7661846435967
4 575.7431234707183 575.7431234707183
8 282.21136564731336 282.21136564731336
16 203.68973076129703 203.68973076129706


In [20]:
gamma = 16
gamma = np.longdouble(gamma)
cashflows = cashflows.astype(np.longdouble)
gamma_m1 = gamma - 1.0
gamma_m1_inverse = 1.0 / gamma_m1
gamma_m1_inverse

0.06666666666666666667

In [21]:
u = gamma_m1_inverse - 1.0 / (gamma_m1 * cashflows ** gamma_m1)
u

array([0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666666, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667],
      dtype=float128)

In [22]:
u_mean = np.cumsum(u)/indices
u

array([0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666666, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667],
      dtype=float128)

In [23]:
ce = 1.0 / (1.0 - gamma_m1 * u_mean) ** gamma_m1_inverse
ce

<ipython-input-23-b9e78fc1a9eb>:1: RuntimeWarning: divide by zero encountered in true_divide
  ce = 1.0 / (1.0 - gamma_m1 * u_mean) ** gamma_m1_inverse


array([        inf,         inf,         inf,         inf, 19.24840058,
       17.88902061,  7.68531085,  7.75403178,  7.8151576 ,  7.87024548,
        3.56446585,  3.58520254,  3.60438496,  3.6222366 ,  3.63893554,
        3.65462604,  3.66942662,  3.68343587,  3.6967367 ,  3.70939952,
        3.72148465,  3.73304413,  3.74412323,  3.75476155,  3.76499392,
        3.77485119,  3.78436075,  3.7935471 ,  3.8024322 ,  3.81100937],
      dtype=float128)

In [25]:
# test numerical precision
# if we get to 100x Bengen = 12000 we're definitely OK, 20x = 2400 probably OK
# fails first around gamma = 20
maxgamma = 64
failed = False
for gamma in range(maxgamma+1):
    for calibration in np.logspace(0, 5, 11):
        test_series = np.ones(30) * calibration
        # can I distinguish a change in 1 unit per year
        difference = crra_ce(test_series + 1, gamma) - crra_ce(test_series, gamma)
        if np.isfinite(difference) and difference > 0:  # could also check approximately 30
            continue
        else:
            print(gamma, calibration, difference)
            failed = True
            break
    if not failed:
        print(gamma, "OK")


0 OK
1 OK
2 OK
3 OK
4 OK
5 OK
6 OK
7 OK
8 OK
9 OK
10 100000.0 0.0
11 100000.0 0.0
12 100000.0 0.0
13 31622.776601683792 0.0
14 31622.776601683792 0.0
15 31622.776601683792 nan
16 31622.776601683792 nan
17 31622.776601683792 nan
18 31622.776601683792 nan
19 31622.776601683792 0.0
20 10000.0 0.0
21 10000.0 0.0
22 10000.0 0.0
23 10000.0 0.0
24 10000.0 0.0
25 10000.0 0.0
26 10000.0 0.0
27 10000.0 0.0
28 10000.0 nan
29 10000.0 nan
30 10000.0 nan
31 10000.0 nan
32 10000.0 0.0
33 10000.0 nan
34 10000.0 0.0
35 10000.0 nan
36 10000.0 0.0
37 10000.0 0.0
38 10000.0 0.0
39 10000.0 nan
40 10000.0 0.0
41 10000.0 0.0
42 10000.0 0.0
43 10000.0 0.0
44 3162.2776601683795 0.0
45 3162.2776601683795 0.0
46 10000.0 0.0
47 3162.2776601683795 0.0
48 3162.2776601683795 nan
49 3162.2776601683795 0.0
50 3162.2776601683795 nan
51 3162.2776601683795 0.0
52 3162.2776601683795 nan
53 3162.2776601683795 0.0
54 3162.2776601683795 nan
55 3162.2776601683795 nan
56 3162.2776601683795 nan
57 3162.2776601683795 nan
58 3162

<ipython-input-13-b38b14863696>:30: RuntimeWarning: invalid value encountered in longdouble_scalars
  ce = 1.0 / (1.0 - gamma_m1 * u) ** gamma_m1_inverse
<ipython-input-13-b38b14863696>:30: RuntimeWarning: divide by zero encountered in longdouble_scalars
  ce = 1.0 / (1.0 - gamma_m1 * u) ** gamma_m1_inverse


In [ ]:
# it's worth noting that when gamma > 1
# lim(U) as x->infinity is bounded by 1/gamma
# in other words as wealth increases, utility increases, by less and less and there is a hard limit utility never exceeds
# some people find this highly objectionable on philosophical grounds. for one thing, for sufficiently high gamma there are some bets you will never take because in utility terms they would need a higher utility payoff than hard limit
# for some size bet, no bet will be accepted regardless of the odds, because the utility risked is more than the maximum utility available to game
# on the other hand, any bet accepted at any level will always be acepted at the same % of wealth for any level of wealth
# I'm not particularly sure how I feel about that, other than it creates thorny philosophical and numerical issues and I want to stay away from thorny philosophical and numerical problems. 
# numerically you want to scale your inputs so that implicitly the hard limit beyond which utility increases are too small to measure is e.g. billions and not hundreds of dollars.
# important to be cognizant of that hard limit because once you get in its vicinity increases in utility are very small and you run into numerical problems
